In [1]:
import sys
sys.path.append('/Users/iSam/Codes/Bridge/')

from evaluation.metrics import  mean_explain_variance
from util.misc import get_folds
from voxel_preprocessing.preprocess_voxels import detrend
from voxel_preprocessing.preprocess_voxels import minus_average_resting_states
from voxel_preprocessing.preprocess_voxels import reduce_mean
from voxel_preprocessing.select_voxels import VarianceFeatureSelection
from voxel_preprocessing.select_voxels import TopkFeatureSelection
from language_preprocessing.tokenize import SpacyTokenizer

import tensorflow as tf
import numpy as np
import itertools
from tqdm import tqdm
import os
import pickle

from ExplainBrain import ExplainBrain
from read_dataset.harrypotter_data import HarryPotterReader
from computational_model.text_encoder import TfHubElmoEncoder
from mapping_models.sk_mapper import SkMapper
import tensorflow as tf

/Users/iSam/anaconda/envs/python3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
FLAGS = tf.flags.FLAGS

tf.flags.DEFINE_integer('subject_id', 1, 'subject id')
tf.flags.DEFINE_integer('fold_id', 0, 'fold id')
tf.flags.DEFINE_list('delays',[-8,-6,-4,-2,0,2,4,6,8] , 'delay list')
tf.flags.DEFINE_boolean('cross_delay', True, 'try different train and test delays')

tf.flags.DEFINE_float('alpha', 1.0, 'alpha')
tf.flags.DEFINE_string('embedding_dir', None, 'path to the file containing the embeddings')
tf.flags.DEFINE_string('brain_data_dir', '/Users/samiraabnar/Codes/Data/harrypotter/', 'Brain Data Dir')


tf.flags.DEFINE_string('embedding_type', 'elmo', 'type of embedding')
tf.flags.DEFINE_string('context_mode', 'sentence', 'type of context')
tf.flags.DEFINE_integer('past_window', 0, 'window size to the past')
tf.flags.DEFINE_integer('future_window', 0, 'window size to the future')

tf.flags.DEFINE_boolean('save_data', True ,'save data')
tf.flags.DEFINE_boolean('load_data', True ,'load data')

tf.flags.DEFINE_boolean('f', None ,'notebook flag')

In [3]:
hparams = FLAGS

# Define how we want to read the brain data
print("1. initialize brain data reader ...")
brain_data_reader = HarryPotterReader(data_dir=hparams.brain_data_dir)

# Define how we want to computationaly represent the stimuli
print("2. initialize text encoder ...")
stimuli_encoder = TfHubElmoEncoder(hparams)

print("3. initialize mapper ...")
mapper = (SkMapper, {'hparams':hparams})

# Build the pip
# eline object
print("4. initialize Explainer...")
explain_brain = ExplainBrain(hparams, brain_data_reader, stimuli_encoder, mapper)

1. initialize brain data reader ...
2. initialize text encoder ...
INFO:tensorflow:Using /var/folders/mm/vdv49cwx09d7t8vzqgngsgh00000gn/T/tfhub_modules to cache modules.
INFO:tensorflow:Initialize variable module/bilm/char_embed:0 from checkpoint b'/var/folders/mm/vdv49cwx09d7t8vzqgngsgh00000gn/T/tfhub_modules/9bb74bc86f9caffc8c47dd7b33ec4bb354d9602d/variables/variables' with bilm/char_embed
INFO:tensorflow:Initialize variable module/bilm/CNN/W_cnn_0:0 from checkpoint b'/var/folders/mm/vdv49cwx09d7t8vzqgngsgh00000gn/T/tfhub_modules/9bb74bc86f9caffc8c47dd7b33ec4bb354d9602d/variables/variables' with bilm/CNN/W_cnn_0
INFO:tensorflow:Initialize variable module/bilm/CNN/b_cnn_0:0 from checkpoint b'/var/folders/mm/vdv49cwx09d7t8vzqgngsgh00000gn/T/tfhub_modules/9bb74bc86f9caffc8c47dd7b33ec4bb354d9602d/variables/variables' with bilm/CNN/b_cnn_0
INFO:tensorflow:Initialize variable module/bilm/CNN/W_cnn_1:0 from checkpoint b'/var/folders/mm/vdv49cwx09d7t8vzqgngsgh00000gn/T/tfhub_modules/9bb74bc8